In [3]:
import numpy as np

In [462]:
ts1 = np.loadtxt("ts1.txt")
ts2 = np.loadtxt("ts2.txt")
ts3 = np.loadtxt("ts3.txt")
ts4 = np.loadtxt("ts4.txt")

In [4]:
from Operations.ST_LocalExtrema import ST_LocalExrema as STLE

In [1]:
from ctypes import *
import ctypes

In [7]:
so_file = "/Users/joshua/Desktop/MS_shannon.so"
lib = CDLL(so_file)

In [8]:
lib.entropy.argtypes = [
    np.ctypeslib.ndpointer(dtype=np.float64, flags='C_CONTIGUOUS'),
    ctypes.c_int,
    ctypes.c_int,
    ctypes.c_int,
    ctypes.POINTER(ctypes.c_double)
]
lib.entropy.restype = None  # void function

In [9]:
def shannon_entropy(data, bin_count, depth):
    """
    Calculate Shannon Entropy using the C function.
    
    Args:
    data (numpy.array): Input time series data
    bin_count (int): Number of bins for encoding
    depth (int): Depth of the encoding
    
    Returns:
    float: Calculated Shannon Entropy
    """
    data = np.ascontiguousarray(data, dtype=np.float64)
    length = len(data)
    result = ctypes.c_double()
    
    lib.entropy(data, length, bin_count, depth, ctypes.byref(result))
    
    return result.value

In [10]:
shannon_entropy(ts3, 2, 5)

3.258990526199341

In [12]:
numBins = 10
numBins = [numBins]

In [4]:
from Operations.CO_Embed2_Shapes import CO_Embed2_Shapes as coes

In [22]:
import numpy as np
from Operations.CO_HistogramAMI import CO_HistogramAMI
from Operations.CO_FirstCrossing import CO_FirstCrossing
from Operations.IN_AutoMutualInfo import IN_AutoMutualInfo
from Operations.CO_AutoCorr import CO_AutoCorr
from PeripheryFunctions.BF_SignChange import BF_SignChange
from PeripheryFunctions.BF_iszscored import BF_iszscored
from scipy.optimize import curve_fit
import warnings

def CO_AddNoise(y, tau = 1, amiMethod = 'even', extraParam = None, randomSeed = None):
    """
    CO_AddNoise: Changes in the automutual information with the addition of noise

    Parameters:
    y (array-like): The input time series (should be z-scored)
    tau (int or str): The time delay for computing AMI (default: 1)
    amiMethod (str): The method for computing AMI:
                      'std1','std2','quantiles','even' for histogram-based estimation,
                      'gaussian','kernel','kraskov1','kraskov2' for estimation using JIDT
    extraParam: e.g., the number of bins input to CO_HistogramAMI, or parameter for IN_AutoMutualInfo
    randomSeed (int): Settings for resetting the random seed for reproducible results

    Returns:
    dict: Statistics on the resulting set of automutual information estimates
    """

    if not BF_iszscored(y):
        warnings.warn("Input time series should be z-scored")``
    
    # Set tau to minimum of autocorrelation function if 'ac' or 'tau'
    if tau in ['ac', 'tau']:
        tau = CO_FirstCrossing(y, 'ac', 0, 'discrete')
    
    # Generate noise
    if randomSeed is not None:
        np.random.seed(randomSeed)
    noise = np.random.randn(len(y)) # generate uncorrelated additive noise

    # Set up noise range
    noiseRange = np.linspace(0, 3, 50) # compare properties across this noise range
    numRepeats = len(noiseRange)

    # Compute the automutual information across a range of noise levels
    amis = np.zeros(numRepeats)
    if amiMethod in ['std1', 'std2', 'quantiles', 'even']:
        # histogram-based methods using my naive implementation in CO_Histogram
        for i in range(numRepeats):
            amis[i] = CO_HistogramAMI(y + noiseRange[i]*noise, tau, amiMethod, extraParam or 10) # use default numBins if None
            if np.isnan(amis[i]):
                raise ValueError('Error computing AMI: Time series too short (?)')
    if amiMethod in ['gaussian','kernel','kraskov1','kraskov2']:
        for i in range(numRepeats):
            amis[i] = IN_AutoMutualInfo(y + noiseRange[i]*noise, tau, amiMethod, extraParam)
            if np.isnan(amis[i]):
                raise ValueError('Error computing AMI: Time series too short (?)')
    
    # Output statistics
    out = {}
    # Proportion decreases
    out['pdec'] = np.sum(np.diff(amis) < 0) / (numRepeats - 1)

    # Mean change in AMI
    out['meanch'] = np.mean(np.diff(amis))

    # Autocorrelation of AMIs
    out['ac1'] = CO_AutoCorr(amis, 1, 'Fourier')[0]
    out['ac2'] = CO_AutoCorr(amis, 2, 'Fourier')[0]

    # Noise level required to reduce ami to proportion x of its initial value
    firstUnderVals = [0.75, 0.50, 0.25]
    for val in firstUnderVals:
        out[f'firstUnder{val*100}'] = firstUnder_fn(val * amis[0], noiseRange, amis)

    # AMI at actual noise levels: 0.5, 1, 1.5 and 2
    noiseLevels = [0.5, 1, 1.5, 2]
    for nlvl in noiseLevels:
        out[f'ami_at_{nlvl*10}'] = amis[np.argmax(noiseRange >= nlvl)]

    # Count number of times the AMI function crosses its mean
    out['pcrossmean'] = np.sum(np.diff(np.sign(amis - np.mean(amis))) != 0) / (numRepeats - 1)

    # Fit exponential decay
    expFunc = lambda x, a, b : a * np.exp(b * x)
    popt, pcov = curve_fit(expFunc, noiseRange, amis, p0=[amis[0], -1])
    out['fitexpa'], out['fitexpb'] = popt
    residuals = amis - expFunc(noiseRange, *popt)
    ss_res = np.sum(residuals**2)
    ss_tot = np.sum((amis - np.mean(amis))**2)
    out['fitexpr2'] = 1 - (ss_res / ss_tot)
    out['fitexpadjr2'] = 1 - (1-out['fitexpr2'])*(len(amis)-1)/(len(amis)-2-1)
    out['fitexprmse'] = np.sqrt(np.mean(residuals**2))

    # Fit linear function
    p = np.polyfit(noiseRange, amis, 1)
    out['fitlina'], out['fitlinb'] = p
    lin_fit = np.polyval(p, noiseRange)
    out['linfit_mse'] = np.mean((lin_fit - amis)**2)

    return out

# helper functions
def firstUnder_fn(x, m, p):
    """
    Find the value of m for the first time p goes under the threshold, x. 
    p and m vectors of the same length
    """
    first_i = next((m_val for m_val, p_val in zip(m, p) if p_val < x), m[-1])
    return first_i


In [10]:
def _calculate_intervals(elements, scales):
    return np.array([int((elements / (1 << scale)) + 0.5) for scale in range(scales - 1, -1, -1)])

In [25]:
def dfa(x, intervals):
    elements = len(x)
    flucts = np.zeros(len(intervals))

    for scale, interval in enumerate(intervals):
        subdivs = int(np.ceil(elements / interval))
        trend = np.zeros(elements)

        for i in range(subdivs):
            start = i * interval
            end = start + interval
            if end > elements:
                trend[start:] = x[start:]
                break
            segment = x[start:end]
            t = np.arange(interval)
            coeffs = np.polyfit(t, segment, 1)
            trend[start:end] = np.polyval(coeffs, t)
        flucts[scale] = np.sqrt(np.mean((x - trend)**2))

    return flucts

In [12]:
def fastdfa(x, intervals=None):
    """
    Perform fast detrended fluctuation analysis on a nonstationary input signal.

    Args:
    x: Input signal (must be a 1D numpy array)
    intervals: Optional list of sample interval widths at each scale

    Returns:
    intervals: List of sample interval widths at each scale
    flucts: List of fluctuation amplitudes at each scale
    """
    if x.ndim != 1:
        raise ValueError("Input sequence must be a vector.")
    
    elements = len(x)
    
    if intervals is None:
        scales = int(np.log2(elements))
        if (1 << (scales - 1)) > elements / 2.5:
            scales -= 1
        intervals = _calculate_intervals(elements, scales)
    else:
        if len(intervals) < 2:
            raise ValueError("Number of intervals must be greater than one.")
        if np.any((intervals > elements) | (intervals < 3)):
            raise ValueError("Invalid interval size: must be between size of sequence x and 3.")
    
    y = np.cumsum(x)
    flucts = dfa(y, intervals)
    
    return intervals, flucts

In [30]:
xpts, ypts = fastdfa(ts1)

In [31]:
coeffs = np.polyfit(np.log10(xpts), np.log10(ypts), 1)

In [32]:
coeffs

array([ 0.61481407, -0.93273621])

In [37]:
from Operations.SC_fastdfa import SC_fastdfa

In [38]:
SC_fastdfa(ts1)

0.614814074638312

In [75]:
def _buffer(X, n, p=0, opt=None):
    '''Mimic MATLAB routine to generate buffer array

    MATLAB docs here: https://se.mathworks.com/help/signal/ref/buffer.html.
    Taken from: https://stackoverflow.com/questions/38453249/does-numpy-have-a-function-equivalent-to-matlabs-buffer 

    Parameters
    ----------
    x: ndarray
        Signal array
    n: int
        Number of data segments
    p: int
        Number of values to overlap
    opt: str
        Initial condition options. default sets the first `p` values to zero,
        while 'nodelay' begins filling the buffer immediately.

    Returns
    -------
    result : (n,n) ndarray
        Buffer array created from X
    '''
    import numpy as np

    if opt not in [None, 'nodelay']:
        raise ValueError('{} not implemented'.format(opt))

    i = 0
    first_iter = True
    while i < len(X):
        if first_iter:
            if opt == 'nodelay':
                # No zeros at array start
                result = X[:n]
                i = n
            else:
                # Start with `p` zeros
                result = np.hstack([np.zeros(p), X[:n-p]])
                i = n-p
            # Make 2D array and pivot
            result = np.expand_dims(result, axis=0).T
            first_iter = False
            continue

        # Create next column, add `p` results from last col if given
        col = X[i:i+(n-p)]
        if p != 0:
            col = np.hstack([result[:,-1][-p:], col])
        i += n-p

        # Append zeros if last row and not length `n`
        if len(col) < n:
            col = np.hstack([col, np.zeros(n-len(col))])

        # Combine result with next row
        result = np.hstack([result, np.expand_dims(col, axis=0).T])

    return result

In [440]:
import numpy as np
from Operations.CO_AutoCorr import CO_AutoCorr
from warnings import warn
from scipy.interpolate import CubicSpline
import statsmodels.api as sm

def SC_FluctAnal(x, q = 2, wtf = 'rsrange', tauStep = 1, k = 1, lag = None, logInc = True):
    """
    """

    N = len(x) # time series length

    # Compute integrated sequence
    if lag is None or lag == 1:
        y = np.cumsum(x) # normal cumulative sum
    else:
        y = np.cumsum(x[::lag]) # if a lag is specified, do a decimation
    
    # Perform scaling over a range of tau, up to a fifth the time-series length
    #-------------------------------------------------------------------------------
    # Peng (1995) suggests 5:N/4 for DFA
    # Caccia suggested from 10 to (N-1)/2...
    #-------------------------------------------------------------------------------

    if logInc:
        # in this case tauStep is the number of points to compute
        if tauStep == 1:
            # handle the case where tauStep is 1, but we want to take the upper 
            # limit (MATLAB) rather than the lower (Python)
            tauStep += 1
            logRange = np.linspace(np.log(5), np.log(np.floor(N/2)), tauStep)[1] # take the second entry (upper limit)
        else:
            logRange = np.linspace(np.log(5), np.log(np.floor(N/2)), tauStep)
        taur = np.unique(np.round(np.exp(logRange)).astype(int))
    else:
        taur = np.arange(5, int(np.floor(N/2)) + 1, tauStep)
    ntau = len(taur) # % analyze the time series across this many timescales
    #print(taur)
    if ntau < 8: # fewer than 8 points
        # time series is too short for analysing using this fluctuation analysis. 
        warn(f"This time series (N = {N}) is too short to analyze using this fluctuation analysis.")
        out = np.NaN
    
    # 2) Compute the fluctuation function, F
    F = np.zeros(ntau)
    # each entry corresponds to a given scale, tau
    for i in range(ntau):
        # buffer the time series at the scale tau
        tau = taur[i]
        y_buff = _buffer(y, tau)
        if y_buff.shape[1] > int(np.floor(N/tau)): # zero-padded, remove trailing set of pts...
            y_buff = y_buff[:, :-1]

        # analyzed length of time series (with trailing pts removed)
        nn = y_buff.shape[1] * tau

        if wtf == 'nothing':
            y_dt = y_buff.reshape(nn, 1)
        elif wtf == 'endptdiff':
            # look at differences in end-points in each subsegment
            y_dt = y_buff[-1, :] - y_buff[0, :]
        elif wtf == 'range':
            y_dt = np.ptp(y_buff, axis=0)
        elif wtf == 'std':
            y_dt = np.std(y_buff, ddof=1, axis=0)
        elif wtf == 'iqr':
            y_dt = np.percentile(y_buff, 75, method='hazen', axis=0) - np.percentile(y_buff, 25, method='hazen', axis=0)
        elif wtf == 'dfa':
            tt = np.arange(1, tau + 1)[:, np.newaxis]
            for j in range(y_buff.shape[1]):
                p = np.polyfit(tt.ravel(), y_buff[:, j], k)
                y_buff[:, j] -= np.polyval(p, tt).ravel()
            y_dt = y_buff.reshape(-1)
        elif wtf == 'rsrange':
            # Remove straight line first: Caccia et al. Physica A, 1997
            # Straight line connects end points of each window:
            b = y_buff[0, :]
            m = y_buff[-1, :] - b
            y_buff -= (np.linspace(0, 1, tau)[:, np.newaxis] * m + b)
            y_dt = np.ptp(y_buff, axis=0)
        elif wtf == 'rsrangefit':
            # polynomial fit (order k) rather than endpoints fit: (~DFA)
            tt = np.arange(1, tau + 1)[:, np.newaxis]
            for j in range(y_buff.shape[1]):
                p = np.polyfit(tt.ravel(), y_buff[:, j], k)
                y_buff[:, j] -= np.polyval(p, tt).ravel()
            y_dt = np.ptp(y_buff, axis=0)
        else:
            raise ValueError(f"Unknown fluctuation analysis method '{wtf}")
        
        F[i] = (np.mean(y_dt**q))**(1/q)

    # Smooth unevenly-distributed points in log space
    if logInc:
        logtt = np.log(taur)
        logFF = np.log(F)
        numTimeScales = ntau
    else:
        # need to smooth the unevenly-distributed pts (using a spline)
        logtaur = np.log(taur)
        logF = np.log(F)
        numTimeScales = 50 # number of sampling pts across the range
        logtt = np.linspace(np.min(logtaur), np.max(logtaur), numTimeScales) # even sampling in tau
        # equivalent to spline function in MATLAB
        spl_fit = CubicSpline(logtaur, logF)
        logFF = spl_fit(logtt)

    # Linear fit the log-log plot: full range
    out = {}
    out = doRobustLinearFit(out, logtt, logFF, range(numTimeScales), '')
    
    """ 
    WE NEED SOME SORT OF AUTOMATIC DETECTION OF GRADIENT CHANGES/NUMBER
    %% OF PIECEWISE LINEAR PIECES

    ------------------------------------------------------------------------------
    Try assuming two components (2 distinct scaling regimes)
    ------------------------------------------------------------------------------
    Move through, and fit a straight line to loglog before and after each point.
    Find point with the minimum sum of squared errors

    First spline interpolate to get an even sampling of the interval
    (currently, in the log scale, there are relatively more at slower timescales)

    Determine the errors
    """
    sserr = np.full(numTimeScales, np.nan) # don't choose the end pts
    minPoints = 6
    for i in range(minPoints, (numTimeScales-minPoints)+1):
        r1 = np.arange(i)
        p1 = np.polyfit(logtt[r1], logFF[r1], 1) # first degree polynomial
        r2 = np.arange(i-1, numTimeScales)
        p2 = np.polyfit(logtt[r2], logFF[r2], 1)
        sserr[i] = (np.linalg.norm(np.polyval(p1, logtt[r1]) - logFF[r1]) +
                    np.linalg.norm(np.polyval(p2, logtt[r2]) - logFF[r2]))
    
    # breakPt is the point where it's best to fit a line before and another line after
    breakPt = np.nanargmin(sserr)
    r1 = np.arange(breakPt)
    r2 = np.arange(breakPt-1, numTimeScales)

    # Proportion of the domain of timescales corresponding to the first good linear fit
    out['prop_r1'] = len(r1)/numTimeScales
    out['logtausplit'] = logtt[breakPt-1]
    out['ratsplitminerr'] = np.nanmin(sserr) / out['ssr']
    out['meanssr'] = np.nanmean(sserr)
    out['stdssr'] = np.nanstd(sserr, ddof=1)

    # Check that at least 3 points are available
    # Now we perform the robust linear fitting and get statistics on the two segments
    # R1
    out = doRobustLinearFit(out, logtt, logFF, r1, 'r1_')
    # R2
    out = doRobustLinearFit(out, logtt, logFF, r2, 'r2_')

    if np.isnan(out['r1_alpha']) or np.isnan(out['r2_alpha']):
        out['alpharat'] = np.nan
    else:
        out['alpharat'] = out['r1_alpha'] / out['r2_alpha']

    return out

In [434]:
def doRobustLinearFit(out, logtt, logFF, theRange, fieldName):
    """
    Get robust linear fit statistics on scaling range
    Adds fields to the output structure
    """
    if len(theRange) < 8 or np.all(np.isnan(logFF[theRange])):
        out[f'{fieldName}linfitint'] = np.nan
        out[f'{fieldName}alpha'] = np.nan
        out[f'{fieldName}se1'] = np.nan
        out[f'{fieldName}se2'] = np.nan
        out[f'{fieldName}ssr'] = np.nan
        out[f'{fieldName}resac1'] = np.nan
    else:
        X = sm.add_constant(logtt[theRange])
        model = sm.RLM(logFF[theRange], X, M=sm.robust.norms.TukeyBiweight())
        results = model.fit()
        out[f'{fieldName}linfitint'] = results.params[0]
        out[f'{fieldName}alpha'] = results.params[1]
        out[f'{fieldName}se1'] = results.bse[0]
        out[f'{fieldName}se2'] = results.bse[1]
        out[f'{fieldName}ssr'] = np.mean(results.resid ** 2)
        out[f'{fieldName}resac1'] = CO_AutoCorr(results.resid, 1, 'Fourier')[0]
    
    return out
    

In [489]:
import numpy as np
from warnings import warn 

def SSC_MMA(y, doOverlap = False, scaleRange = None, qRange = None):
    """
    Python implementation of multiscale multifractal analysis (MMA)
    
    Parameters:
    -----------
    y : array_like
        Input time series
    do_overlap : bool, optional
        Whether to use overlapping windows (default is False)
    scale_range : tuple, optional
        (min_scale, max_scale) for analysis (default is None)
    q_range : tuple, optional
        (q_min, q_max) for analysis (default is None)
    
    Returns:
    --------
    dict
        Dictionary containing various MMA statistics
    """

    N = len(y)

    if scaleRange is None:
        scaleRange = [10, np.ceil(N/40)]

    minScale = scaleRange[0]
    maxScale = scaleRange[1]
    if (maxScale/5) < minScale:
        warn(f"Time-series (N={N}) too short for multiscale multifractal analysis")
        out = np.NaN
    elif maxScale % 5 != 0:
        maxScale = np.ceil(maxScale/5)*5
    
    if qRange is None:
        qRange = [-5, 5]
    qMin, qMax = qRange
    stepVal = 0.1

    qList = np.arange(start=qMin, stop=qMax+stepVal, step=stepVal)
    qList[qList == 0] = 0.0001

    prof = np.cumsum(y)
    numIncrements = 20

    sListFull = np.unique(np.round(np.linspace(minScale, maxScale, numIncrements)))
    fqs = []
    for s in sListFull:
        if doOverlap:
            coordinates = np.array([np.arange(i, i+s) for i in range(len(prof)-s+1)])
        else:            
            coordinates = [prof[i:i+int(s)] for i in range(0, len(prof)-int(s)+1, int(s))]
        print(coordinates)
        segments = prof[coordinates]
        x_base = np.arange(1, s+1)
        f2nis = []

        for seg in segments:
            fit = np.polyfit(x_base, seg, 2)
            variance = np.mean((seg - np.polyval(fit, x_base))**2)
            f2nis.append(variance)
        
        f2nis = np.array(f2nis)

        for q in qList:
            fqs.append([q, s, (np.mean(f2nis**(q/2)))**(1/q)])
        
    fqs = np.array(fqs)
    fqsll = np.column_stack((fqs[:, 0], fqs[:, 1], np.log(fqs[:, 1]), np.log(fqs[:, 2])))
    
    print(fqs)
    


In [935]:
import numpy as np

def CO_TranslateShape(y, shape = 'circle', d = 2, howToMove = 'pts'):
    """
    """
    N = len(y)

    # add a time index
    ty = np.column_stack((np.arange(1, N+1), y))
    #-------------------------------------------------------------------------------
    # Generate the statistics on the number of points inside the shape as it is
    # translated across the time series
    #-------------------------------------------------------------------------------
    if howToMove == 'pts':
        if shape == 'circle':
            r = d
            w = int(np.floor(r))
            rnge = np.arange(1 + w, N - w)
            NN = len(rnge)
            np_array = np.zeros(NN)

            for i in range(NN):
                win = ty[rnge[i] - w:rnge[i] + w + 1, :]
                difwin = win - np.ones((2*w+1, 1)) * ty[rnge[i], :]
                np_array[i] = np.sum(np.sum(difwin**2, axis=1) <= r**2)
           
        elif shape == 'rectangle':
            w = d
            rnge = np.arange(1 + d, N - d)
            NN = len(rnge)
            np_array = np.zeros(NN)
            for i in range(NN):
                np_array[i] = np.sum(np.abs(y[rnge[i] - d:rnge[i] + d + 1]) <= np.abs(y[rnge[i]]))
        else:
            raise ValueError(f"Unknown shape '{shape}'")
    else:
        raise ValueError(f"Unknown setting for howToMove: '{howToMove}'")
    
    # statistics on the number of hits inside in the shape
    out = {}
    out['max'] = np.max(np_array)
    out['std'] = np.std(np_array, ddof=1)
    out['mean'] = np.mean(np_array)

    # Count the hits
    unique, counts = np.unique(np_array, return_counts=True)
    histnp = dict(zip(unique, counts))

    # Compute the mode of the histogram
    mode_val = max(histnp, key=histnp.get)
    out['npatmode'] = histnp[mode_val] / NN
    out['mode'] = mode_val

    for i in range(1, min(12, 2*w+2)):
        out[f"{['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'eleven'][i-1]}s"] \
            = np.mean(np_array == i)
    
    # Stationarity of the statistics in 2,3, & 4 segments of the time series:
    # NOT YET IMPLEMENTED - DEPENDS ON SY_SLIDINGWINDOW

    return out


In [936]:
CO_TranslateShape(ts1, shape='circle')

{'max': 3.0,
 'std': 0.0,
 'mean': 3.0,
 'npatmode': 1.0,
 'mode': 3.0,
 'ones': 0.0,
 'twos': 0.0,
 'threes': 1.0,
 'fours': 0.0,
 'fives': 0.0}

In [938]:
from scipy import stats

In [951]:
kde_fit = stats.gaussian_kde(ts1, bw_method='scott')
xi = np.linspace(np.min(ts1), np.max(ts1), 100)
f = kde_fit(xi)
f

array([0.47077312, 0.49536432, 0.51661102, 0.53418911, 0.54789987,
       0.55767468, 0.56357235, 0.56576941, 0.5645444 , 0.56025776,
       0.55332889, 0.54421224, 0.53337411, 0.52127155, 0.5083346 ,
       0.4949524 , 0.48146371, 0.46815152, 0.4552416 , 0.4429043 ,
       0.4312588 , 0.42037921, 0.4103016 , 0.40103139, 0.3925506 ,
       0.38482445, 0.37780715, 0.3714467 , 0.36568874, 0.36047937,
       0.35576716, 0.35150451, 0.34764829, 0.34416021, 0.34100676,
       0.33815899, 0.33559221, 0.33328549, 0.33122131, 0.32938508,
       0.32776477, 0.3263505 , 0.32513424, 0.32410943, 0.3232708 ,
       0.32261401, 0.32213551, 0.32183233, 0.32170191, 0.32174202,
       0.32195068, 0.32232616, 0.32286702, 0.32357218, 0.32444108,
       0.32547386, 0.32667158, 0.32803644, 0.3295721 , 0.33128393,
       0.33317933, 0.33526801, 0.33756231, 0.34007753, 0.34283227,
       0.34584873, 0.34915306, 0.35277569, 0.3567515 , 0.36112002,
       0.36592527, 0.37121548, 0.37704223, 0.38345913, 0.39051

In [952]:
def FitKernelSmooth(x):
    
    m = np.mean(x)
    kdefit = stats.gaussian_kde(x)

In [953]:
import antropy as ant


In [960]:
ant.perm_entropy(ts2, order=3, delay=2)

2.4299028461250582

In [1014]:
import antropy as ant
from antropy.utils import _embed, _xlogx
from math import factorial
import numpy as np
from Operations.CO_FirstCrossing import CO_FirstCrossing

def _perm_entropy_all(x, order=3, delay=1, normalize=False, return_normedCounts=False):
    # compute all relevant perm entropy stats
    if isinstance(delay, (list, np.ndarray, range)):
        return np.mean([_perm_entropy_all(x, order=order, delay=d, normalize=normalize) for d in delay])
    x = np.array(x)
    ran_order = range(order)
    hashmult = np.power(order, ran_order)
    assert delay > 0, "delay must be greater than zero."
    # Embed x and sort the order of permutations
    sorted_idx = _embed(x, order=order, delay=delay).argsort(kind="quicksort")
    # Associate unique integer to each permutations
    hashval = (np.multiply(sorted_idx, hashmult)).sum(1)
    # Return the counts
    _, c = np.unique(hashval, return_counts=True)
    # Use np.true_divide for Python 2 compatibility
    p = np.true_divide(c, c.sum())
    pe = -_xlogx(p).sum()
    if normalize:
        pe /= np.log2(factorial(order))
    
    if return_normedCounts:
        return pe, p
    else:
        return pe

def EN_PermEn(y, m = 2, tau = 1):
    """
    """
    if tau == 'ac':
        tau = CO_FirstCrossing(y, 'ac', 0, 'discrete')
    elif not isinstance(tau, int):
        raise TypeError("Invalid type for tau. Can be either 'ac' or an integer.")
    
    pe, p = _perm_entropy_all(y, order=m, delay=tau, normalize=False, return_normedCounts=True)
    pe_n = ant.perm_entropy(y, order=m, delay=tau, normalize=True)
    Nx = len(y) - (m-1) * tau # get the number of embedding vectors
    # p will only contain non-zero probabilities, so to make the output consistent with MATLAB, we need to add a correction:
    # not saying this is correct, but this is how it is implemented in MATLAB and this is a port...
    lenP = len(p)
    numZeros = factorial(m) - lenP
    # append the zeros to the end of p
    p = np.concatenate([np.array(p), np.zeros(numZeros)])
    p_LE = [np.maximum(1/Nx, p[i]) for i in range(len(p))]
    permEnLE = -np.sum(p_LE * np.log(p_LE))/(m-1)

    out = {}
    out['permEn'] = pe
    out['normPermEn'] = pe_n
    out['permEnLE'] = permEnLE

    return out


In [1018]:
EN_PermEn(ts1, 4, 'ac')

{'permEn': 3.0946435887719645,
 'normPermEn': 0.6749550488766738,
 'permEnLE': 0.7479272237443636}

In [998]:
from Operations.EN_PermEn import EN_PermEn as EPE

In [1016]:
EPE(ts1, 5, 'ye')

TypeError: '>' not supported between instances of 'str' and 'int'

In [1007]:
factorial(5)

120